# Digital Atlas of Australia API IderDown client

This Notebook is a software/service client [SpacePrez](https://github.com/surroundaustralia/Prez/) APIs deployed as part of the [digital Atlas of Australia](https://www.ga.gov.au/scientific-topics/national-location-information/digital-atlas-of-australia)'s data layers. 

This client emulates the behaviour of the [Location Index (LocI)](http://ga.gov.au/locationindex) project's [IderDown](http://loci.cat/iderDown.html) client which was custom software that was able to "discovery and download related Loc-I identifiers for selected spatial features".


### 1. Install dependencies and register useful functions

In [37]:
import httpx
import tabulate


def sparql_query(endpoint, q):
    r = httpx.get(
        endpoint.strip("/") + "/sparql",
        params={"query": q},
        headers={"Accept": "application/sparql-results+json"}
    )
    if r.status_code == 200:
        return [True, r.json()["results"]["bindings"]]
    else:
        return [False, r.status_code, r.text]
    
    
def tableize(json):
    tbl = []
    for d in json: 
        tbl.append([v["value"] for k, v in d.items()])

    table = tabulate.tabulate(tbl, tablefmt="html")
    return table


def get_contained(endpoint, container_iri, feature_collection):
    q = f"""        
        PREFIX geo: <http://www.opengis.net/ont/geosparql#>
        PREFIX geof: <http://www.opengis.net/def/function/geosparql/>
        PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

        SELECT ?f
        WHERE {{
          <{container_iri}> geo:hasGeometry/geo:asWKT ?container .

          <{feature_collection}> rdfs:member ?f .
          ?f geo:hasGeometry/geo:asWKT ?wkt .

          FILTER (geof:sfWithin(?wkt, ?container))
        }}
        LIMIT 5    
        """
    
    r = httpx.get(
        endpoint.strip("/") + "/sparql",
        params={"query": q},
        headers={"Accept": "application/sparql-results+json"},
        timeout=60
    )
    if r.status_code == 200:
        return [True, r.json()["results"]["bindings"]]
    else:
        return [False, r.status_code, r.text]

## 1. Basic IderDown
For a given Feature, get the IDs of all contained Features of another class.

E.g., for the SA3 10201, get all the SA2s

In [39]:
c = get_contained(
    "https://asgs.linked.fsdf.org.au",
    "https://linked.data.gov.au/dataset/asgsed3/SA3/10202", 
    "https://linked.data.gov.au/dataset/asgsed3/SA2"
)
if c[0]:
    display(tableize(c[1]))

https://linked.data.gov.au/dataset/asgsed3/SA2/102021044
https://linked.data.gov.au/dataset/asgsed3/SA2/102021045
https://linked.data.gov.au/dataset/asgsed3/SA2/102021046
https://linked.data.gov.au/dataset/asgsed3/SA2/102021047
https://linked.data.gov.au/dataset/asgsed3/SA2/102021048


...or the SA1s

In [ ]:
c = get_contained(
    "https://asgs.linked.fsdf.org.au",
    "https://linked.data.gov.au/dataset/asgsed3/SA3/10201", 
    "https://linked.data.gov.au/dataset/asgsed3/SA1"
)
if c[0]:
    display(tableize(c[1]))

# Other interesting functions 

## 2. Provide API locations

In [40]:
endpoints = [
    "https://asgs.linked.fsdf.org.au"
]

## 2. Get all Feature Collections from all given endpoints   

In [41]:
q = """
    PREFIX geo: <http://www.opengis.net/ont/geosparql#>
    PREFIX dcterms: <http://purl.org/dc/terms/>

    SELECT *
    WHERE {
      ?fc a geo:FeatureCollection ;
          dcterms:title ?t .
    }
    ORDER BY ?t
    """

fcs = []
for endpoint in endpoints:
    qr = sparql_query(endpoint, q)
    if qr[0]:
        # add in endpoint
        for row in qr[1]:
            extended_row = {**{"endpoint": {"value": endpoint}}, **row}
            fcs.append(extended_row)
tableize(fcs)

https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/AUS,Australia Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/GCCSA,Greater Capital City Statistical Areas
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/MB,Mesh Block Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/STE,States & Territories Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/SA1,Statistical Areas Level 1 Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/SA2,Statistical Areas Level 2 Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/SA3,Statistical Areas Level 3 Feature Collection
https://asgs.linked.fsdf.org.au,https://linked.data.gov.au/dataset/asgsed3/SA4,Statistical Areas Level 4 Feature Collection


## 3. Get first 10 Features from a selected Feature Collection

- select an endpoint and a Feature Collection from the table generated above

In [42]:
endpoint = "https://asgs.linked.fsdf.org.au"
fc = "https://linked.data.gov.au/dataset/asgsed3/SA3"
q = f"""
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

    SELECT *
    WHERE {{
      <{fc}> rdfs:member ?f .
      OPTIONAL {{?f dcterms:title|rdfs:label ?t}}
    }}
    ORDER BY ?t
    LIMIT 10
    """
qr = sparql_query(endpoint, q)
if qr[0]:
    display(tableize(qr[1]))

https://linked.data.gov.au/dataset/asgsed3/SA3/40101,Adelaide City
https://linked.data.gov.au/dataset/asgsed3/SA3/40102,Adelaide Hills
https://linked.data.gov.au/dataset/asgsed3/SA3/50901,Albany
https://linked.data.gov.au/dataset/asgsed3/SA3/10901,Albury
https://linked.data.gov.au/dataset/asgsed3/SA3/70201,Alice Springs
https://linked.data.gov.au/dataset/asgsed3/SA3/50601,Armadale
https://linked.data.gov.au/dataset/asgsed3/SA3/11001,Armidale
https://linked.data.gov.au/dataset/asgsed3/SA3/12501,Auburn
https://linked.data.gov.au/dataset/asgsed3/SA3/50101,Augusta - Margaret River - Busselton
https://linked.data.gov.au/dataset/asgsed3/SA3/30201,Bald Hills - Everton Park
